## Initial Dependencies and modules

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import RobertaTokenizer, RobertaModel
from transformers import AutoTokenizer, AutoModelForMaskedLM

## Importing dataset

In [4]:
kinn_train_path = 'KINNEWS_train.csv'
kinn_test_path = 'KINNEWS_test.csv'
embd_path =  'W2V-Kin-100.txt'

## Dataset preprocessing

In [22]:
import pandas as pd


df = pd.read_csv(kinn_train_path)


# Concatenating title and content
separator = " [SEP] "
df['text'] = df['title'] + separator + df['content']
df = df.drop(columns=['title', 'content'])

print(df.head())


   label                                               text
0      3  bugesera nyuma y’inzara yaharanzwe karahahira ...
1      5  mutzig beer fest itegerejwe n’abantu benshi ki...
2      2  abakinnyi bamagare batanu berekeje isiganwa ny...
3      3  kireheumugabo afungiwe gufatanwa imiti irimo c...
4      7  sobanukirwa internet g izahindura ubuzima munt...


In [23]:
# Making the labels 0 based (0 to 13 instead of 1 to 14)
df['label'] = df['label'] - 1
df.head()

,label,text
0,2,bugesera nyuma y’inzara yaharanzwe karahahira ...
1,4,mutzig beer fest itegerejwe n’abantu benshi ki...
2,1,abakinnyi bamagare batanu berekeje isiganwa ny...
3,2,kireheumugabo afungiwe gufatanwa imiti irimo c...
4,6,sobanukirwa internet g izahindura ubuzima munt...


In [24]:
#Train-val spit
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'].astype(str), df['label'], test_size=0.2, random_state=42)

In [25]:
#Tokenizing inputs
tokenizer = AutoTokenizer.from_pretrained("dsfsi/BantuBERTa")

# Tokenize the input texts
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=128)

In [26]:
#Defining the dataset for model training
class KinyarwandaDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [27]:
train_dataset = KinyarwandaDataset(train_encodings, train_labels.tolist())
val_dataset = KinyarwandaDataset(val_encodings, val_labels.tolist())

## Training on Kinyarwanda

In [41]:
# import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("dsfsi/BantuBERTa", num_labels=14).to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='/Users/casarulez/Projects/DDSI/BantuBERT/results',          # Output directory
    num_train_epochs=8,              # Number of training epochs
    per_device_train_batch_size=32,   # Batch size for training
    per_device_eval_batch_size=32,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
    load_best_model_at_end=True,
    evaluation_strategy='steps',
    no_cuda=True                     # Disable CUDA
)

# Function to check dataset shapes
def check_dataset_shapes(dataset):
    for i, data in enumerate(dataset):
        inputs, labels = data
        print(f"Batch {i} - inputs shape: {inputs.shape}, labels shape: {labels.shape}")
        if i == 2:  # Check first few batches
            break


# Create Trainer instance
trainer = Trainer(
    model=model,                         # The instantiated 🤗 Transformers model to be trained
    args=training_args,                  # Training arguments, defined above
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=val_dataset,            # Evaluation dataset
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at dsfsi/BantuBERTa and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/casarulez/anaconda3/envs/ddsi/lib/python3.11/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/casarulez/anaconda3/envs/ddsi/lib/python3.11/site-packages/transformers/training_args.py:1509: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [34]:
# Train the model
trainer.train()

Step,Training Loss,Validation Loss
10,2.705700,2.694502
20,2.679000,2.661948
30,2.646700,2.607794
40,2.577700,2.536743
50,2.506000,2.448467
60,2.419700,2.346152
70,2.331700,2.242808
80,2.215000,2.149000
90,2.146800,2.073106
100,2.068300,2.011152


KeyboardInterrupt: 

## Resuming training from 500 steps checkpoint

In [42]:
trainer.train(resume_from_checkpoint='/Users/casarulez/Projects/DDSI/BantuBERT/results/checkpoint-500')

Step,Training Loss,Validation Loss
510,0.513100,0.572419
520,0.584400,0.576919
530,0.520600,0.618766
540,0.432200,0.573591
550,0.601400,0.566575
560,0.579700,0.578899
570,0.526200,0.550460
580,0.426900,0.533034
590,0.455000,0.653348
600,0.537600,0.544219


KeyboardInterrupt: 

## Preparing the test dataset

In [7]:
# Load the test dataset
df_test = pd.read_csv(kinn_test_path)

separator = " [SEP] "
df_test['text'] = df_test['title'] + separator + df_test['content']

df_test = df_test.drop(columns=['title', 'content'])
df_test['label'] = df_test['label'] - 1

print(df_test.head())


   label                                               text
0      1  ikipe y’ u rwanda amavubi yahesheje u rwanda a...
1     10  urubyiruko itorero erc giterane cy’ububyutse k...
2      3  rusizi bambaye udupfukamunwa n’ubwo bamwe bata...
3      4  abanyarwanda batatu begukanye ibihembo pam awa...
4     10  light family choir igiye gukora igitaramo cy’a...


## Loading the model from the optimum checkpoint and testing metrics - Kinyarwanda

In [14]:
from transformers import AutoTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("dsfsi/BantuBERTa")

# Load the model from the checkpoint
checkpoint_path = '/Users/casarulez/Projects/DDSI/BantuBERT/results/checkpoint-1000'
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)


In [15]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
from tqdm import tqdm  # Import tqdm

# Set a maximum sequence length that matches the model's expected input size
#max_length = 512  # You can adjust this based on your model's maximum input length

# Tokenize the test data
encodings = tokenizer(list(df_test['text']), truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
labels = torch.tensor(df_test['label'].values)

# Create a DataLoader
dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], labels)
dataloader = DataLoader(dataset, batch_size=32)

# Evaluate the model
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in tqdm(dataloader, desc="Processing Batches"):
        input_ids, attention_mask, label_ids = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(label_ids.cpu().numpy())

# Calculate accuracy and F1 score
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")


Processing Batches:   0%|                               | 0/133 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Processing Batches: 100%|█████████████████████| 133/133 [07:00<00:00,  3.16s/it]

Accuracy: 0.8601
F1 Score: 0.8555


## Random prediction - Kinyarwanda

In [ ]:
num_samples = 5
random_indices = np.random.choice(len(df_test), num_samples, replace=False)

# Extract texts and labels for these indices
sample_texts = df_test.iloc[random_indices]['text'].tolist()
sample_labels = df_test.iloc[random_indices]['label'].tolist()

# Tokenize the sample texts
encodings = tokenizer(list(df_test['text']), truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')

# Prepare inputs for the model
input_ids = encodings['input_ids']
attention_mask = encodings['attention_mask']
labels = torch.tensor(sample_labels)

# Make predictions
model.eval()
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=-1).cpu().numpy()

# Print the results
for idx, (text, true_label, pred) in enumerate(zip(sample_texts, sample_labels, preds)):
    print(f"Sample {idx + 1}:")
    print(f"Text: {text}")
    print(f"True Label: {true_label}")
    print(f"Predicted Label: {pred}")
    print()

## Loading the Kirundi test dataset

In [5]:
# Load the test dataset
kir_test = pd.read_csv('/Users/casarulez/Projects/DDSI/KIRNEWS/cleaned/test.csv')

separator = " [SEP] "
kir_test['text'] = kir_test['title'] + separator + kir_test['content']

kir_test = kir_test.drop(columns=['title', 'content'])
kir_test['label'] = kir_test['label'] - 1

print(kir_test.head())


   label                                               text
0      1  intamba m’urugamba z’abakobwa zaryoheye abarun...
1      0  unviriza hano ikiganiro c’umukuru w’igihugu ya...
2     13  ibintu bidasanzwe wokorera umukunzi wawe bikam...
3     12  na nyamwema bitwararitswe bokwiga nk’abandi [S...
4      0  sentare ijejwe kubahiriza ibwirizwa nshingiro ...


## Loading the model from the optimum checkpoint and testing metrics - Kirundi (Before fine tuning)

In [9]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

checkpoint_path = '/Users/casarulez/Projects/DDSI/BantuBERT/results/checkpoint-1000'

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dsfsi/BantuBERTa")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

In [12]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
from tqdm import tqdm  # Import tqdm

# Tokenize the test data
max_length=512
encodings = tokenizer(list(kir_test['text']), truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
labels = torch.tensor(kir_test['label'].values)

# Create a DataLoader
dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], labels)
dataloader = DataLoader(dataset, batch_size=32)

# Evaluate the model
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in tqdm(dataloader, desc="Processing Batches"):
        input_ids, attention_mask, label_ids = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(label_ids.cpu().numpy())

# Calculate accuracy and F1 score
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")


Processing Batches: 100%|███████████████████████| 29/29 [01:28<00:00,  3.06s/it]

Accuracy: 0.7454
F1 Score: 0.7375


## Random prediction - Kirundi

In [75]:
num_samples = 5
random_indices = np.random.choice(len(df_test), num_samples, replace=False)

# Extract texts and labels for these indices
sample_texts = kir_test.iloc[random_indices]['text'].tolist()
sample_labels = kir_test.iloc[random_indices]['label'].tolist()

# Tokenize the sample texts
encodings = tokenizer(sample_texts, truncation=True, padding=True, return_tensors='pt')

# Prepare inputs for the model
input_ids = encodings['input_ids']
attention_mask = encodings['attention_mask']
labels = torch.tensor(sample_labels)

# Make predictions
model.eval()
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=-1).cpu().numpy()

# Print the results
for idx, (text, true_label, pred) in enumerate(zip(sample_texts, sample_labels, preds)):
    print(f"Sample {idx + 1}:")
    print(f"Text: {text}")
    print(f"True Label: {true_label}")
    print(f"Predicted Label: {pred}")
    print()

Sample 1:
Text: sadio man ngo bazogerageza gutsinda fc barcelona [SEP] mugihe hasigaye indwi gusa kugira liverpool yakirwe fc barcelona i camp nou mugice ca kane gishira icanyuma champions league sadio man umunya sngal akinira liverpool avugako bazogerageza gutsinda fc barcelona nubwo umurwi ukomeye fc barcelone numwe m mirwi myiza kuriyi isi ariko tuzogerageza tubatsinde ivyo vyandikwa n’ikinyamakuru la vie sngalaise ni mugihe barcelona imaze inkino idatsindirwa i camp nou kw’itariki rusama mwaka liverpool izoba ifise igikorwa gikomeye
True Label: 1
Predicted Label: 1

Sample 2:
Text: ikiziga c’umukinyi mpuzamakungu w’umurundi papy fatty cashikanywe kibuga c’indege a bujumbura [SEP] inyuma y’indwi zitatu umukinyi mpuzamakungu w’umurundi yakinira umurwi nserukira gihugu w’uburundi mupira w’amaguruintamba mu rugamba papy fatty asandabiye mu kibuga kubera indwara y’umutima mu gihe yariko arakinira umurwi wiwe malanti chiefs mu cicaro ca mbere mu gihugu a eswatini cahoze citwa swaziland k

# Fine tuning on Kirundi

## Loading the Kirundi training dataset

In [13]:
# Load the test dataset
kir_train = pd.read_csv('/Users/casarulez/Projects/DDSI/KIRNEWS/cleaned/train.csv')

separator = " [SEP] "
kir_train['text'] = kir_train['title'] + separator + kir_train['content']

kir_train = kir_train.drop(columns=['title', 'content'])
kir_train['label'] = kir_train['label'] - 1

print(kir_test.head())

   label                                               text
0      1  intamba m’urugamba z’abakobwa zaryoheye abarun...
1      0  unviriza hano ikiganiro c’umukuru w’igihugu ya...
2     13  ibintu bidasanzwe wokorera umukunzi wawe bikam...
3     12  na nyamwema bitwararitswe bokwiga nk’abandi [S...
4      0  sentare ijejwe kubahiriza ibwirizwa nshingiro ...


In [14]:
#Train-val spit
train_texts, val_texts, train_labels, val_labels = train_test_split(kir_train['text'].astype(str), kir_train['label'], test_size=0.2, random_state=42)

In [15]:
#Tokenizing inputs
tokenizer = AutoTokenizer.from_pretrained("dsfsi/BantuBERTa")

# Tokenize the input texts
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=128)

In [16]:
#Defining the dataset for model training
class KirundiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [17]:
train_dataset = KirundiDataset(train_encodings, train_labels.tolist())
val_dataset = KirundiDataset(val_encodings, val_labels.tolist())

In [20]:
#Defining the training parameters
import torch

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")


model = model = AutoModelForSequenceClassification.from_pretrained('/Users/casarulez/Projects/DDSI/BantuBERT/results/checkpoint-1000', num_labels=14).to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='/Users/casarulez/Projects/DDSI/BantuBERT/results/kirundi',          # Output directory
    num_train_epochs=8,              # Number of training epochs
    per_device_train_batch_size=32,   # Batch size for training
    per_device_eval_batch_size=32,    # Batch size for evaluation
    warmup_steps=100,                # Number of warmup steps
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
    load_best_model_at_end=True,
    evaluation_strategy='steps',
    no_cuda=True                     # Disable CUDA
)

# Create Trainer instance
trainer = Trainer(
    model=model,                         # The instantiated 🤗 Transformers model to be trained
    args=training_args,                  # Training arguments, defined above
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=val_dataset,            # Evaluation dataset
)

In [21]:
# Train the model
trainer.train()

Step,Training Loss,Validation Loss
10,1.171100,1.025996
20,1.012400,0.929528
30,0.851000,0.863899
40,0.797400,0.773900
50,0.714400,0.715769
60,0.631000,0.706702
70,0.681200,0.642085
80,0.649200,0.622976
90,0.580000,0.700473
100,0.505500,0.584709


RuntimeError: [enforce fail at inline_container.cc:595] . unexpected pos 217431296 vs 217431184

In [26]:
checkpoint_path = '/Users/casarulez/Projects/DDSI/BantuBERT/results/kirundi/checkpoint-500'

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dsfsi/BantuBERTa")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

In [27]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
from tqdm import tqdm  # Import tqdm

# Tokenize the test data
max_length=512
encodings = tokenizer(list(kir_test['text']), truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
labels = torch.tensor(kir_test['label'].values)

# Create a DataLoader
dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], labels)
dataloader = DataLoader(dataset, batch_size=32)

# Evaluate the model
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in tqdm(dataloader, desc="Processing Batches"):
        input_ids, attention_mask, label_ids = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(label_ids.cpu().numpy())

# Calculate accuracy and F1 score
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Processing Batches: 100%|███████████████████████| 29/29 [01:27<00:00,  3.03s/it]

Accuracy: 0.8657
F1 Score: 0.8606
